In [2]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- TensorFlow Import Handling ---
HAS_TF = False
TF_ERROR_MSG = ""

try:
    import tensorflow as tf
    HAS_TF = True
except ImportError as e:
    TF_ERROR_MSG = str(e)

# --- Define the Generator ---
def build_generator(latent_dim):
    """
    Generator: Takes random noise (latent_dim) and generates a 28x28 grayscale image.
    """
    if not HAS_TF:
        raise ImportError("TensorFlow is not available. Please install TensorFlow to use this function.")
    
    model = tf.keras.Sequential(name="Generator")
    
    # Input layer - latent space
    model.add(tf.keras.layers.Dense(256, input_dim=latent_dim, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.01))
    
    # Hidden layers to increase dimensionality
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.01))
    
    model.add(tf.keras.layers.Dense(1024, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU(alpha=0.01))
    
    # Output layer - reshape to 28x28 image (pixel values -1 to 1)
    model.add(tf.keras.layers.Dense(28 * 28 * 1, activation='tanh'))
    model.add(tf.keras.layers.Reshape((28, 28, 1)))
    
    return model

# --- Define the Discriminator ---
def build_discriminator(img_shape):
    """
    Discriminator: Takes an image (28x28) and classifies it as Real (1) or Fake (0).
    """
    if not HAS_TF:
        raise ImportError("TensorFlow is not available. Please install TensorFlow to use this function.")
    
    model = tf.keras.Sequential(name="Discriminator")
    
    # Flatten the image to a 1D vector
    model.add(tf.keras.layers.Flatten(input_shape=img_shape))
    
    # Hidden dense layers
    model.add(tf.keras.layers.Dense(512))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.01))
    
    model.add(tf.keras.layers.Dense(256))
    model.add(tf.keras.layers.LeakyReLU(alpha=0.01))
    
    # Output layer - Probability (0 to 1)
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model

# --- Interactive UI ---
class GANArchitect:
    def __init__(self):
        self.output = widgets.Output()
        
        self.latent_dim_slider = widgets.IntSlider(
            value=100, min=10, max=200, step=10, 
            description='Latent Dim:', continuous_update=False
        )
        
        self.btn_build = widgets.Button(
            description="Build Models",
            button_style='primary',
            icon='cogs'
        )
        self.btn_build.on_click(self.show_architecture)
        
        # Initial display
        self.show_architecture(None)

    def show_architecture(self, _):
        with self.output:
            clear_output(wait=True)
            
            if not HAS_TF:
                print("="*60)
                print("ERROR: TensorFlow could not be imported.")
                print("="*60)
                print(f"Error message: {TF_ERROR_MSG}")
                print("\nPlease make sure TensorFlow is installed and configured correctly.")
                print("For more information, visit: https://www.tensorflow.org/install")
                print("="*60)
                return
            
            latent_dim = self.latent_dim_slider.value
            img_shape = (28, 28, 1)
            
            # Build models
            generator = build_generator(latent_dim)
            discriminator = build_discriminator(img_shape)
            
            # Display Generator Summary
            print("="*60)
            print(f"1. GENERATOR MODEL (Input: Vector of size {latent_dim})")
            print("="*60)
            generator.summary()
            
            print("\n")
            
            # Display Discriminator Summary
            print("="*60)
            print(f"2. DISCRIMINATOR MODEL (Input: Image of shape {img_shape})")
            print("="*60)
            discriminator.summary()
            
            # Test a forward pass
            noise = np.random.normal(0, 1, (1, latent_dim))
            gen_img = generator.predict(noise, verbose=0)
            decision = discriminator.predict(gen_img, verbose=0)
            
            print("\n" + "="*60)
            print("TEST RUN:")
            print(f"Generated Image Shape: {gen_img.shape}")
            print(f"Discriminator Probability (Fake=0, Real=1): {decision[0][0]:.4f}")
            print("="*60)

    def render(self):
        display(widgets.HTML("<h2>🏭 Simple GAN Architecture Builder (TensorFlow)</h2>"))
        display(widgets.VBox([
            self.latent_dim_slider,
            self.btn_build,
            self.output
        ]))

# Run App
app = GANArchitect()
app.render()

HTML(value='<h2>🏭 Simple GAN Architecture Builder (TensorFlow)</h2>')